In [1]:
!pip install -q tiktoken rank_bm25 openai langchain pypdf chroma faiss-cpu langchain-community unstructured unstructured-inference sentence-transformers
!pip install -q langchain-text-splitters nltk fastembed chromadb pyarrow langchain-openai peft langchain_experimental fuzzywuzzy
!pip install -q -U FlagEmbedding
!pip install --upgrade --quiet langchain-openai langchainhub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.schema import Document

from langchain_openai import OpenAI
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from sentence_transformers import SentenceTransformer


from langchain.vectorstores import FAISS, Chroma
from langchain.retrievers import BM25Retriever
from langchain.vectorstores.pinecone import Pinecone

from langchain.chains import RetrievalQA
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import nltk
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
import os

from google.colab import drive

model = SentenceTransformer('all-MiniLM-L6-v2')
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Mounted at /content/drive


In [3]:
os.environ["OPENAI_API_KEY"]=""

In [13]:
path = '/content/drive/MyDrive/kdt_240424/Team Project/2차/RAG_Accident_Case_General.csv'
df = pd.read_csv(path, encoding='utf-8-sig')

In [15]:
import json

# 데이터프레임을 JSON 형식으로 변환 (각 행을 JSON 객체로 변환)
json_data = df.to_dict(orient='records')

# JSON 파일로 저장
json_file_path = 'RAG_Accident_Case_General.json'

with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [17]:
json_data

[{'사고유형': '보1',
  '사고상황': '보행자A : 적색에 횡단 개시, 적색에 충격 / 자동차B : 녹색에 교차로 진입',
  '사고내용': '신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 규제를 받는 우회전)이 보행자신호등 적색신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 적색신호가 여전히 켜져 있는 상황에서 충격한 사고이다.',
  '과실비율': 'A70 : B30',
  '과실해설': '도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에 따른 일시정지 의무가 있는 점을 고려하여 보행자의 기본과실을 70%로 정하였다.',
  '관련법규': '도로교통법 제5조(신호 또는 지시에 따를 의무) / 도로교통법 제27조(보행자의 보호) / 도로교통법 제48조(안전운전 및 친환경 경제운전의 의무) / 도로교통법 시행규칙 별표2(신호기가 표시하는 신호의 종류 및 신호의 뜻) / 도로교통법 시행규칙 별표5(신호등의 신호 순서[제7조 제2항 관련])',
  '판결사례': '대법원 1990.8.10. 선고 90도1116 판결. 횡단보도의 표지판이나 신호대가 설치되어 있지는 않으나 도로의 바닥에 페인트로 횡단보도 표시를 하여 놓은 곳으로써 피고인이 진행하는 반대 차선쪽은 오래되어 거의 지워진 상태이긴 하나 피고인이 운행하는 차선쪽은 횡단보도인 점을 식별할 수 있을 만큼 그 표시가 되어있는 곳에서 교통사고가 난 경우에는 교통사고가 도로교통법상 횡단보도상에서 일어난것으로 인정된다. / 대법원 1993.2.23. 선고 92도2077 판결. 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대차선상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무

In [ ]:
def split_text_recursive(text, chunk_size=500, chunk_overlap=200):
    # RecursiveCharacterTextSplitter 인스턴스 생성
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = recursive_splitter.split_text(text)
    return chunks

# 텍스트 분할: RecursiveCharacterTextSplitter 사용
recursive_chunks = split_text_recursive(combined_text, chunk_size=1000, chunk_overlap=100)

documents_recursive = [Document(page_content=chunk) for chunk in recursive_chunks]

------------------------------------------------------------------------------

In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [6]:
def concatenate_columns(df):
    # 모든 칼럼의 텍스트를 하나의 문자열로 결합
    texts = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)
    return "\n\n".join(texts)

combined_text = concatenate_columns(df)

In [7]:
combined_text

'보1 보행자A : 적색에 횡단 개시, 적색에 충격 / 자동차B : 녹색에 교차로 진입 신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 규제를 받는 우회전)이 보행자신호등 적색신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 적색신호가 여전히 켜져 있는 상황에서 충격한 사고이다. A70 : B30 도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에 따른 일시정지 의무가 있는 점을 고려하여 보행자의 기본과실을 70%로 정하였다. 도로교통법 제5조(신호 또는 지시에 따를 의무) / 도로교통법 제27조(보행자의 보호) / 도로교통법 제48조(안전운전 및 친환경 경제운전의 의무) / 도로교통법 시행규칙 별표2(신호기가 표시하는 신호의 종류 및 신호의 뜻) / 도로교통법 시행규칙 별표5(신호등의 신호 순서[제7조 제2항 관련]) 대법원 1990.8.10. 선고 90도1116 판결. 횡단보도의 표지판이나 신호대가 설치되어 있지는 않으나 도로의 바닥에 페인트로 횡단보도 표시를 하여 놓은 곳으로써 피고인이 진행하는 반대 차선쪽은 오래되어 거의 지워진 상태이긴 하나 피고인이 운행하는 차선쪽은 횡단보도인 점을 식별할 수 있을 만큼 그 표시가 되어있는 곳에서 교통사고가 난 경우에는 교통사고가 도로교통법상 횡단보도상에서 일어난것으로 인정된다. / 대법원 1993.2.23. 선고 92도2077 판결. 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대차선상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다. https://accident.knia.or.kr/video/man1.mp4\n\n보2 보행자A : 적색에 횡단 시작, 적

In [11]:
def split_text_recursive(text, chunk_size=500, chunk_overlap=200):
    # RecursiveCharacterTextSplitter 인스턴스 생성
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = recursive_splitter.split_text(text)
    return chunks

# 텍스트 분할: RecursiveCharacterTextSplitter 사용
recursive_chunks = split_text_recursive(combined_text, chunk_size=1000, chunk_overlap=100)

documents_recursive = [Document(page_content=chunk) for chunk in recursive_chunks]

In [12]:
documents_recursive

[Document(page_content='보1 보행자A : 적색에 횡단 개시, 적색에 충격 / 자동차B : 녹색에 교차로 진입 신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 규제를 받는 우회전)이 보행자신호등 적색신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 적색신호가 여전히 켜져 있는 상황에서 충격한 사고이다. A70 : B30 도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에 따른 일시정지 의무가 있는 점을 고려하여 보행자의 기본과실을 70%로 정하였다. 도로교통법 제5조(신호 또는 지시에 따를 의무) / 도로교통법 제27조(보행자의 보호) / 도로교통법 제48조(안전운전 및 친환경 경제운전의 의무) / 도로교통법 시행규칙 별표2(신호기가 표시하는 신호의 종류 및 신호의 뜻) / 도로교통법 시행규칙 별표5(신호등의 신호 순서[제7조 제2항 관련]) 대법원 1990.8.10. 선고 90도1116 판결. 횡단보도의 표지판이나 신호대가 설치되어 있지는 않으나 도로의 바닥에 페인트로 횡단보도 표시를 하여 놓은 곳으로써 피고인이 진행하는 반대 차선쪽은 오래되어 거의 지워진 상태이긴 하나 피고인이 운행하는 차선쪽은 횡단보도인 점을 식별할 수 있을 만큼 그 표시가 되어있는 곳에서 교통사고가 난 경우에는 교통사고가 도로교통법상 횡단보도상에서 일어난것으로 인정된다. / 대법원 1993.2.23. 선고 92도2077 판결. 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대차선상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다. https://accident.knia.or.kr/video/man1.mp4'),

===========================================================================================================

In [ ]:
# Import the necessary libraries
from FlagEmbedding import BGEM3FlagModel

class BGEM3ColBERTEmbeddings:
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

    def encode(self, sentences):
        output = self.model.encode(sentences, return_dense=True, return_sparse=True, return_colbert_vecs=True)
        return output['colbert_vecs']

    def compute_similarity(self, vecs1, vecs2):
        return [self.model.colbert_score(v1, v2) for v1 in vecs1 for v2 in vecs2]

# Instantiate the ColBERT embeddings
bgem3_colbert_embeddings = BGEM3ColBERTEmbeddings()

class BGEM3DenseEmbeddings:
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

    def encode(self, sentences):
        output = self.model.encode(sentences, batch_size=12, max_length=8192)
        return output['dense_vecs']

    def compute_similarity(self, vecs1, vecs2):
        # Compute similarity matrix using dot product
        return vecs1 @ vecs2.T

# Instantiate the dense embeddings
bgem3_dense_embeddings = BGEM3DenseEmbeddings()

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

imgs/.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
from langchain.embeddings.base import Embeddings

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=True)

    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=True)

In [ ]:
split_methods = {
    "Recursive": [Document(page_content=chunk) for chunk in recursive_chunks]
}

# 임베딩 방법 정의
embedding_methods = {
    "OpenAIEmbeddings": OpenAIEmbeddings(),
}

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [ ]:
from fuzzywuzzy import fuzz

# 서치 방법 (FAISS, Chroma 등)
def create_vectorstore(documents, embedding_method):
    vectorstores = {}
    if isinstance(embedding_method, BGEM3DenseEmbeddings):
        dense_embeddings = embedding_method.encode([doc.page_content for doc in documents])
        vectorstores["FAISS"] = FAISS.from_embeddings(dense_embeddings)
    elif isinstance(embedding_method, BGEM3ColBERTEmbeddings):
        colbert_embeddings = embedding_method.encode([doc.page_content for doc in documents])
        vectorstores["FAISS"] = FAISS.from_embeddings(colbert_embeddings)
    else:
        vectorstores["FAISS"] = FAISS.from_documents(documents, embedding=embedding_method)
    return vectorstores

# 리트리버 생성 방법
def create_retrievers(documents, vectorstores, embedding_method):
    retrievers = {}
    if isinstance(embedding_method, BGEM3DenseEmbeddings):
        dense_embeddings = embedding_method.encode([doc.page_content for doc in documents])
        vectorstores["FAISS"] = FAISS.from_embeddings(dense_embeddings)
    elif isinstance(embedding_method, BGEM3ColBERTEmbeddings):
        colbert_embeddings = embedding_method.encode([doc.page_content for doc in documents])
        vectorstores["FAISS"] = FAISS.from_embeddings(colbert_embeddings)
    else:
        retrievers["FAISS with OpenAIEmbeddings"] = vectorstores["FAISS"].as_retriever()
    return retrievers

# Retriever를 테스트하는 함수
def test_retriever(query, retriever):
    # query가 dict인 경우 query 부분만 추출
    if isinstance(query, dict):
        query = query.get('query', '')
    return retriever.get_relevant_documents(query)

In [ ]:
def calculate_similarity(answer1: str, answer2: str) -> int:
    """
    두 개의 답변 사이의 유사도를 계산합니다.
    """
    return fuzz.partial_ratio(answer1, answer2)

def test_retriever_with_accuracy(query, retriever, correct_answer):
    try:
        # Retriever를 사용하여 관련 문서 검색
        query_results = test_retriever(query, retriever)

        # 검색된 결과가 없으면 빈 리스트 반환
        if not query_results:
            return "No results found", 0

        # 가장 높은 유사도 값을 가진 결과를 선택
        best_result = max(query_results, key=lambda x: calculate_similarity(x.page_content, correct_answer))
        best_result_content = best_result.page_content[:200]  # 결과 내용의 일부만 저장

        # 유사도 계산
        similarity = calculate_similarity(best_result.page_content, correct_answer)
        return best_result_content, similarity

    except Exception as e:
        return f"Error: {e}", 0

def run_all_tests(test_cases):
    global results  # 전역 결과 리스트 사용
    results = []  # 결과 리스트 초기화

    for split_name, split_documents in split_methods.items():
        for embedding_name, embedding_method in embedding_methods.items():
            print(f"=== Testing Split: {split_name}, Embedding: {embedding_name} ===\n")

            # 벡터 스토어 생성
            vectorstores = create_vectorstore(split_documents, embedding_method)

            # Retriever 딕셔너리 생성
            retrievers = create_retrievers(split_documents, vectorstores, embedding_method)

            for example in test_cases:
                query = example['query']
                correct_answer = example['correct_answer']

                print(f"Query: {query}\n")
                for name, retriever in retrievers.items():
                    print(f"Using {name}:")
                    try:
                        # 결과를 테스트하고 정확도 계산
                        result_content, similarity = test_retriever_with_accuracy(query, retriever, correct_answer)
                        result_dict = {
                            'embedding_name': embedding_name,
                            'split_name': split_name,
                            'retriever_name': name,
                            'query': query,
                            'result_content': result_content,
                            'similarity': f"{similarity}%"
                        }
                        results.append(result_dict)  # 결과를 리스트에 추가
                    except Exception as e:
                        print(f"Error with {name}: {e}")
                        results.append({
                            'embedding_name': embedding_name,
                            'split_name': split_name,
                            'retriever_name': name,
                            'query': query,
                            'result_content': f"Error: {e}",
                            'similarity': 0,
                        })
                    print("\n" + "-"*50 + "\n")

    df_results = pd.DataFrame(results)
    df_results.to_csv('rag_csv_test_result.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 테스트 실행
test_cases = [
    {
        "query": "사고유형 '보1'의 과실비율은 어떻게 되나요?",
        "correct_answer": "A70 : B30"
    },
    {
        "query": "신호기가 없는 횡단보도에서 횡단보도를 통과하려는 차량이 횡단보도를 건너고 있던 보행자를 충격한 사고의 과실비율은?",
        "correct_answer": "A0 : B100"
    },
    {
        "query": "보행자가 녹색신호에 횡단을 시작했지만 적색신호에 충격을 받은 경우의 과실비율은?",
        "correct_answer": "A10 : B90"
    },
    {
        "query": "사고유형 '보12'의 사고상황을 설명해주세요.",
        "correct_answer": "신호기가 있는 횡단보도를 통과하여 교차로에 진입하려는 차량이 보행자신호등 녹색점멸신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 녹색점멸 신호가 여전히 켜져 있는 상황에서 충격한 사고이다."
    },
    {
        "query": "보행자신호가 적색일 때 횡단을 시작한 보행자와 녹색신호에 직진하는 차량이 충돌한 경우의 과실비율은?",
        "correct_answer": "A70 : B30"
    },
    {
        "query": "사고유형 '거42'의 사고상황이 뭐야?",
        "correct_answer": "자전거횡단도를 횡단하는 A자전거와 직진하여 자전거횡단도를 통과하는 B차량이 충돌한사고"
    },
    {
        "query": "신호기에 의해 교통정리가 이루어지고 있는 교차로에서 적색신호에 직진하는 A 차량과 교차로에서 녹색화살표 신호에 좌회전하는 B차량이 충돌한 사고의 과실비율은?",
        "correct_answer": "100:0"
    },
    {
        "query": "애니메이션 https://accident.knia.or.kr/video/car15-1.mp4 의 사고상황이 뭐야?",
        "correct_answer": "신호기에 의해 교통정리가 이루어지고 있지 않는 교차로에서 직진하는 A차량과 맞은편 주 행방향에서 좌회전하는 B차량이 충돌한 사고"
    },
    {
        "query": "앞지르기(추월)가 금지된 도로에서 선행하는 A차량과 동일방향에서 후행하다가 A차량을 추월하면서 A차량의 전방으로 진입(앞지르기) 하려는 B차량이 충돌한 사고의 관련법규가 뭐야?",
        "correct_answer": "도로교통법 제20조(진로양보의무) / 도로교통법 제21조(앞지르기 방법 등) / 도로교통법 제60조(갓길 통행금지 등) / 도로교통법 제22조 (앞지르기 금지 장소)"
    },
    {
        "query": "일반 도로의 가장자리(갓길 포함) 또는 도로와 보도의 가장자리에 동시에 걸치고 주·정차 중인 A차량을 동일방향에서 후행하여 진행하는 B차량이 추돌한 사고의 애니메이션은?",
        "correct_answer": "https://accident.knia.or.kr/video/car42-2.mp4"
    },
    {
        "query": "회전 2차로형 이상 회전 교차로 안으로 양 차량이 동시에 진입하면서, 1차로(좌측)에서 진입하는 A차량과 2차로(우측)에서 진입하는 B차량이 충돌한 사고에 대한 판결사례는?",
        "correct_answer": "등록되지 않은 참고판례입니다."
    },
    {
        "query": "2차로형 교차로에서 내부 1차로를 주행하다 2차로로 진출하려는 A차량과 회전교차로로 진입하려는 B차량과의 사고의 판결사례는?",
        "correct_answer": "서울중앙지방법원 2017.12.7.선고 2017나47290 판결. 과실비율은 40:60"
    },
    {
        "query": "사고유형 '거3-4'의 판결사례는?",
        "correct_answer": "서울중앙지방법원 2015.9.10. 선고 2015가단5001249 판결. B차량의 과실 60%."
    },
    {
        "query": "사고유형 '보35'의 과실해설은?",
        "correct_answer": "대법원 2007.7.13. 선고 2007다 26240 판결. 자동차 운전자 과실이 없다. / 서울고등법원 2007.1.18. 선고 2006나40739 판결. B차량 과실20% / 서울지방법원 1999.10.29. 선고 99가단84270 판결. B차량 과실 0%"
    },
    {
        "query": "과실비율이 '70:30'인 사고유형은?",
        "correct_answer": "보1,보8,차8-2,차13-3,차17-2,차8-2,차13-3,차17-2,거10-2,차1-5,차14-1,차20-2,차47-2,차14-1"
    },
    {
        "query": "판결사례가 '등록되지 않은 참고판례입니다.'인 것 중 사고유형이 '차3-8'인 애니메이션은?",
        "correct_answer": "https://accident.knia.or.kr/video/car3-8.mp4"
    },
    {
        "query": "교차로에서 녹색신호에 따라 직진하는 A차량과 상시유턴구역에서 유턴하는 B차량이 충돌한 사고 관련 도로교통법규는 뭘까",
        "correct_answer": "도로교통법 제5조(신호 또는 지시에 따를 의무) / 도로교통법 제18조(횡단 등의 금지) / 도로교통법 제38조(차의 신호)"
    },
    {
        "query": "차량이 직진 중 육교 및 지하도 부근에서 도로를 건너고 있던 보행자를 충격한 사고에 대한 과실해설 해줘",
        "correct_answer": "육교나 지하도가 설치된 경우에는 횡단보도가 설치된 경우보다 운전자가 보행자의 존재를 예견하기 어렵다는 점을 감안하여 보행자의 기본과실을 40%"
    },
    {
        "query": "신호기에 의한 교통정리가 한쪽 방향에만 이루어지고 있는 교차로에서 신호기가 있는 도로를 이용하여 직진하는 B차량과 신호기가 없는 도로를 이용하여 직진(좌회전)하는 A차량 이 충돌한 사고에 대한 과실 비율은?.",
        "correct_answer": "내용 없음"
    },
    {
        "query": " 2차로형 회전교차로에서 회전하다가 우측으로 진출하기 위해 진로변경 중인 A차량과 교차로 내에서 회전 중인 B차량이 충돌한 사고의 과실해설 알려주세요.",
        "correct_answer": "회전교차로에서는 진출을 위하여 언제든지 진로변경을 할 가능성이 높다는 점이 예견가능하고, 또한 같이 회전하고 있는 차량끼리 양보 의무가 가중된다고 할 수 있으므로, 일반적인 진로변경 사고보다 진로변경차량의 과실비율을 약간 낮춰서 양 차량의 기본 과실비율을 60:40으로 정한다."
    },
    {
        "query": "신호기가 있는 횡단보도를 통과하여 교차로에 진입하려는 차량이 보행자신호등 녹색점멸신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 녹색점멸 신호가 여전히 켜져 있는 상황에서 충격한 사고의 사고유형은?",
        "correct_answer": "https://accident.knia.or.kr/video/car3-8.mp4"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car3-5.mp4 인 사고유형은?",
        "correct_answer": "차3-5"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car9-2.png 인 사고내용은?",
        "correct_answer": "신호기에 의해 교통정리가 이루어지고 있지 않고 한쪽에 일시정지 표지가 있는 교차로에 서 일시정지 표지를 위반하여 직진하는 A차량과 일시정지 표지가 없는 교차로에서 우회전 하는 B차량이 충돌한 사고"
    },
    {
        "query": "애니메이션이 https://accident.knia.or.kr/video/car32-1.mp4 의 판결사례는?",
        "correct_answer": "대법원 1992. 7. 28. 선고 92도1137 판결. 자동차의 운전자에게 과실이 있다고 할 수 없다.(교행중 차량의 주의의무) / 서울중앙지방법원 2017. 3. 31. 선고 2016나66409 판결. B과실 50% / 서울중앙지방법원 2017. 7. 9. 선고 2017나58951 판결. B과실 70% / 부산지방법원 2018. 5. 17. 선고 2017나62471 판결. 중앙선침범 B차량 70%, 진출 A차량 30% 인정(주차장 램프구간 교행)"
    },
    {
        "query": "과실해설이 '도로교통법 제26조 제4항에 따라 직진차량이 좌회전차량보다 우선권이 있지만, 신호기 없는 교차로에서의 사고이므로 도표 324를 준용하되 자전거는 통상 저속으로운행하므로 B차량으로서는 이를 발견하여 사고의 발생을 회피할 수 있다는 점 및 자전거는 차량에 비하여 상대방에게 가해의 위험성이 현저히 낮다는 점을 감안하여 양측의 기본과실을 10:90으로 정한다.'인 사고유형은?",
        "correct_answer": "거8-1"
    },
    {
        "query": "사고유형이 '차9-2'인 사고의 과실비율은?",
        "correct_answer": "A80 : B20"
    },
    {
        "query": "사고내용이 신호기에 의해 교통정리가 이루어지고 있지 않는 동일 폭의 교차로에서 좌회전하는 A차량과 A차량의 진행방향 왼쪽 도로에서 좌회전 진입하는 B차량이 충돌한 사고이다. 인 사고의 과실비율은?",
        "correct_answer": "A40 : B60"
    },
    {
        "query": "사고내용이 신호기에 의해 교통정리가 이루어지고 있는 교차로에서 서로 다른 도로를 이용하여 적색신호에 교차로에 진입하여 직진하는 A자전거와 적색신호에 진입하여 직진하는 B차량이충돌한 사고의 사고유형은?",
        "correct_answer": "거1-5"
    },
    {
        "query": "사고유형이 '차53-1'의 애니메이션과 과실비율은?",
        "correct_answer": "https://accident.knia.or.kr/video/car53-1.mp4, A100:B0 "
    },
    {
        "query": "사고내용이 오른쪽 3차로에서 2차로로 진로변경을 하는 A차량과 왼쪽 1차로에서 2차로로 진로변경을 하는 B차량이 충돌한 사고의 과실해설과 과실비율은?",
        "correct_answer": "양 차량이 좌우에서 동시에 진로변경을 하던 중에 발생한 사고로서 양 차량 모두 진로변경방법 위반의 과실이 있는 것이므로 양 차량의 기본 과실비율을 동등하게 보아 50:50으로 정한다, A50 : B50"
    },
]


In [ ]:
run_all_tests(test_cases)

=== Testing Split: Recursive, Embedding: FastEmbedEmbeddings ===

